In [12]:
import sys
sys.path.append('../../')

from bokehComponents.Dashboard import Dashboard
from bokehComponents.BokehComponents import BokehControl,BufferedQueryInterface,BokehTimeseriesGraphic

from bokeh.io import output_notebook,output_file
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.models.widgets import Panel, Tabs #, DataTable, DateFormatter, TableColumn, Tabs, 
from bokeh.models import ColumnDataSource
from bokeh.layouts import layout
from bokeh.plotting import figure,show,output_file
from bokeh.models import Div
from bokeh.models.markers import Circle
from datetime import datetime,timedelta
import pandas_datareader as pdr


In [2]:
from bokeh.plotting import figure, output_file, show

## 4 Custom Graphs
At this point it can be useful to create new types of graphs that are automatically compatible with BufferedDataQueries. In this example we will construct a new kind of graph.

### Status: In Development
Note, the library is in active development, and virtually not method name or class name, or parameter is stable.

## 4.1 Creating Data using the BufferedQueryInterface
At this point, we can give a complete implementation of a well-behaved Buffered Query Interface

In [3]:

class ExampleComplexQuery(BufferedQueryInterface):

    def appendNextAction(self,indicesIn= None):
        #
        ## First, you are given selected data indices. Sometimes this may be null. in this action we will ignore this index.

        assert indicesIn or not indiciesIn # do something with indices if you want

        #
        ## Second, do something with self.data

        next_day = self.current_date+ timedelta(days=3)
        data=pdr.get_data_yahoo('AAPL',self.current_date,next_day)
        self.data['open_list'].append(data['Open'][0])
        self.data['close_list'].append(data['Close'][0])
        self.data['date_time'].append(next_day)
        assert len(self.data['open_list']) == len(self.data['close_list'])
        assert len(self.data['close_list']) == len(self.data['date_time'])

        self.current_date = next_day 

    def deleteAction(self,indicesIn= None):
        #
        ## Remove all the indices, from largest to smallest
        tables = ['open_list','close_list','date_time']
        indicesIn.sort(reverse=True)
        for i in indicesIn:
            for tableName in tables:
                self.data[tableName].pop(i)

    def load_data_buffer(self):
        try:
            if (self.init):
                return
        except:
            pass
        self.init = 1
        start_init=self._settings['start_date']
        end_init=self._settings['end_date']     
        self.current_date = end_init 
        # Grab data
        start=start_init
        end=end_init 

        data=pdr.get_data_yahoo('AAPL',start,end)
        data_sorted=data.sort_index(axis=0,ascending=True)
        date_list=list(data_sorted.index)

        # Save the data elements
        open_list=list(data_sorted['Open'])
        close_list=list(data_sorted['Close'])
        date_time=[datetime.strptime(str(d),'%Y-%m-%d %H:%M:%S').date() for d in date_list]

        #
        ## First, create the data
        self.data = {
            'open_list':open_list,
            'close_list':close_list,
            'date_time':date_time,
            }

        #
        ## Second, register the append action so it can be used.
        self.registerAction('append',ExampleComplexQuery.appendNextAction)
        self.registerAction('delete',ExampleComplexQuery.deleteAction)

### Using the default BokehTimeseriesGraphic
To make a custom graph, the general method is to override the createPlotElement(self,p=PLOT,dd=data_description)

In [21]:
class BarTimeseriesGraphic(BokehTimeseriesGraphic):
    def createPlotElement(self,p,dd):
        #
        ## You can display the data description, if desired, to see what is passed to the object
        #display(dd)
        
        #
        ## All that must be done is the return of a graphic object that is created on the plot
        return p.vbar(x='x', width=0.2, top='y', color="#CAB2D6",source = dd['column_data_source'])
        #return p.line(x='x',y='y',legend=dd['label'],color=dd['color'], alpha=0.5,source = dd['column_data_soruce'])
        
class CircleTimeseriesGraphic(BokehTimeseriesGraphic):
    def createPlotElement(self,p,dd):
        #
        ## You can display the data description, if desired, to see what is passed to the object
        #display(dd)
        
        #
        ## All that must be done is the return of a graphic object that is created on the plot
        glyph = Circle(x='x', line_width=0.2, y='y', line_color="#CAB2D6")
        p.add_glyph(dd['column_data_source'], glyph)       
        return glyph
        #return p.line(x='x',y='y',legend=dd['label'],color=dd['color'], alpha=0.5,source = dd['column_data_soruce'])        

class GraphDashboard(Dashboard):
    def getLayout(self):
        settings = {'title':'Analytics Plot',
                          'width':300,
                          'height':300,
                          'query':ExampleComplexQuery({'start_date':datetime(2017,4,26),
                                               'end_date':datetime(2017,7,26)})}

        settings['data_defs'] = [
                    {'y':'open_list','x':'date_time','key':'open','label':'Open','color':'green' },
                    {'y':'close_list','x':'date_time','key':'close','label':'Close','color':'yellow' }                    
                    ]
        
        self.g = BarTimeseriesGraphic(settings)
        self.c = CircleTimeseriesGraphic(settings)
        l = layout([self.g.getBokehComponent(),[self.c.getBokehComponent()]],sizing_mode='fixed')         
        return l    
    
d = GraphDashboard()
d.showDashboard()

Loading BokehJS ...

Virtually any graph can be created using this method. To see how different graphs are constructed across various datasets please visit Bokeh Documentation on Glypths.

https://bokeh.pydata.org/en/latest/docs/user_guide/plotting.html

Next up, the real fun can begin. It is possible to construt interactive elements. First we will do so in isolation. After, I will explan how interactive components from bokeh automatically work with data sources.